Conversion from .emd to .h5 format.

In [ ]:
# Conversion Function Definitions

from conversion_def import find_and_process_velox_files

In [ ]:
# Convert to HDF5
inputfolder_path= "/home/buster/UOX1/unprocessed/UOX_His_MUA_450nm_spot4_ON_20240311_0928.h5"

find_and_process_velox_files(inputfolder_path)

Peakfinding using peakfinder8 from diffractem package. First open .h5 file in crystfel using .geom file that refers to correct mask file and find good peakfinding settings. Make sure there are enough friedel pairs for center beam finding and fitting. Don't put the mask in the same folder as the .h5 file since it is also .h5 format. If peakfinding has been run old data will be deleted.

In [ ]:
# Peak Finder Function Definitions

from peakfinder_def import find_files_and_run_peakfinding

In [ ]:
# Peak Finder
inputfolder_path = "/home/buster/UOX1/UOX1_min_10_deiced"

# Peakfinder settings:
threshold = 20    
min_snr = 4 
min_pix_count = 2
max_pix_count = 200
local_bg_radius = 10
min_res = 40
max_res = 500
x0 = 512
y0 = 512  

##### Enter parameters above #####

find_files_and_run_peakfinding(inputfolder_path, x0, y0, threshold, min_snr, min_pix_count, max_pix_count, local_bg_radius, min_res, max_res)

Find the center beam spot center on all frames using friedel pairs and  interpolation.

In [ ]:
# Find Center Beam Spot Center Function Definitions

from centerfinding_def import find_files_and_run_centerfinding

In [ ]:
# Find Center Beam Spot Center
tolerance = 8 # If you know that your beam center is close to the center of the detector you can reduce the tolerance to reduce the processing time
min_peaks = 10 # Minimum number of peaks in a frame to be considered
resolution_limit = 200 # Maximum resultuion of peaks to be considered
min_samples_fraction = 0.01  # Fraction of datapoints in the largest cluster to be accepted
x0 = 510
y0 = 516

inputfolder_path= "/home/buster/UOX1"

##### Enter parameters above #####

find_files_and_run_centerfinding(inputfolder_path, x0, y0, tolerance, min_peaks, resolution_limit, min_samples_fraction)

Improve the center beam center with a LOWESS fit

In [ ]:
# Center Beam Drift Refinement Function Definitions

from centerrefinement_def import find_files_and_run_centerrefinement


In [ ]:
# Center Beam Drift Refinement
tolerance = 3 # If you know that your beam center is close to the center of the detector you can reduce the tolerance to reduce the processing time
min_peaks = 10 # Minimum number of peaks in frame to be used for refinement
resolution_limit = 400 # Maximum resolution (pixels) of peaks to be considered for refinement
max_iterations = 20 # Maximum number of iterations for refinement
convergence_threshold = 0.05 # Refinement is stopped when the deviation of the LOWESS fit does not exceed the threshold (given in pixels)

inputfolder_path= "/home/buster/UOX1"

##### Enter your parameters above #####

find_files_and_run_centerrefinement(inputfolder_path, tolerance, min_peaks, resolution_limit, max_iterations, convergence_threshold)

And now to find the indexing using xGandalf (layers = layers of step size )

In [2]:
# Indexing Function Definitions

from gandalf_iterator import gandalf_iterator

In [3]:
# Indexing (To speed up process make sure output stream is in wsl folder)
geomfile_path = "/home/buster/UOX1/UOX1_original/UOX1_original_backup/UOX.geom"
cellfile_path = "/home/buster/UOX1/UOX1_original/UOX1_original_backup/UOX.cell"
input_path =  '/home/buster/UOX1/UOX1_original/UOX1_original_backup'
output_dir = input_path
output_file_base = "UOX1"

num_threads = 23
indexing_method = "xgandalf"
resolution_push = 0
integration_method = "rings"
int_radius = "4,5,8"
min_peaks = 10
xgandalf_tolerance = 0.02
xgandalf_sampling_pitch = 5
xgandalf_iterations = 1
tolerance = "5,5,5,5"
step = 0.5
layers = 2
##### Enter your parameters above #####

gandalf_iterator(geomfile_path, cellfile_path, input_path, output_file_base, output_dir, num_threads, indexing_method, resolution_push, integration_method, int_radius, min_peaks, xgandalf_tolerance, xgandalf_sampling_pitch, xgandalf_iterations, tolerance, step, layers)


Processing XY pairs:   0%|          | 0/25 [00:00<?, ?it/s]WARNING: You did not specify --int-radius.
This is what I understood your unit cell to be:
orthorhombic I, right handed.
a      b      c            alpha   beta  gamma
 80.58  94.49 103.89 A     90.00  90.00  90.00 deg
List of indexing methods:
   0: xgandalf-nolatt-cell      (xgandalf using cell parameters as prior information)
Indexing parameters:
                  Check unit cell parameters: on
                        Check peak alignment: off
                   Refine indexing solutions: off
 Multi-lattice indexing ("delete and retry"): off
                              Retry indexing: on


Running for x=-512, y=-512


240 images processed, 86 hits (35.8%), 48 indexable (55.8% of hits, 20.0% overall), 48 crystals, 47.3 images/sec.
686 images processed, 178 hits (25.9%), 89 indexable (50.0% of hits, 13.0% overall), 89 crystals, 88.2 images/sec.
1098 images processed, 295 hits (26.9%), 154 indexable (52.2% of hits, 14.0% overall), 154 crystals, 80.4 images/sec.
1513 images processed, 389 hits (25.7%), 207 indexable (53.2% of hits, 13.7% overall), 207 crystals, 82.6 images/sec.
2071 images processed, 495 hits (23.9%), 265 indexable (53.5% of hits, 12.8% overall), 265 crystals, 110.9 images/sec.
2378 images processed, 598 hits (25.1%), 322 indexable (53.8% of hits, 13.5% overall), 322 crystals, 61.1 images/sec.
2780 images processed, 710 hits (25.5%), 388 indexable (54.6% of hits, 14.0% overall), 388 crystals, 80.4 images/sec.
3318 images processed, 810 hits (24.4%), 445 indexable (54.9% of hits, 13.4% overall), 445 crystals, 107.3 images/sec.
3690 images processed, 914 hits (24.8%), 507 indexable (55.5%

Running for x=-512.5, y=-512.5


217 images processed, 75 hits (34.6%), 24 indexable (32.0% of hits, 11.1% overall), 24 crystals, 43.1 images/sec.
576 images processed, 160 hits (27.8%), 59 indexable (36.9% of hits, 10.2% overall), 59 crystals, 71.7 images/sec.
1021 images processed, 260 hits (25.5%), 89 indexable (34.2% of hits, 8.7% overall), 89 crystals, 87.5 images/sec.
1183 images processed, 344 hits (29.1%), 122 indexable (35.5% of hits, 10.3% overall), 122 crystals, 32.4 images/sec.
1829 images processed, 431 hits (23.6%), 158 indexable (36.7% of hits, 8.6% overall), 158 crystals, 127.5 images/sec.
2133 images processed, 527 hits (24.7%), 187 indexable (35.5% of hits, 8.8% overall), 187 crystals, 59.8 images/sec.
2469 images processed, 613 hits (24.8%), 216 indexable (35.2% of hits, 8.7% overall), 216 crystals, 67.0 images/sec.
2787 images processed, 714 hits (25.6%), 251 indexable (35.2% of hits, 9.0% overall), 251 crystals, 63.2 images/sec.
3298 images processed, 804 hits (24.4%), 287 indexable (35.7% of hits

...

226 images processed, 76 hits (33.6%), 17 indexable (22.4% of hits, 7.5% overall), 17 crystals, 45.2 images/sec.
579 images processed, 163 hits (28.2%), 38 indexable (23.3% of hits, 6.6% overall), 38 crystals, 68.7 images/sec.
1024 images processed, 261 hits (25.5%), 61 indexable (23.4% of hits, 6.0% overall), 61 crystals, 88.3 images/sec.
1165 images processed, 337 hits (28.9%), 77 indexable (22.8% of hits, 6.6% overall), 77 crystals, 28.1 images/sec.
1828 images processed, 430 hits (23.5%), 102 indexable (23.7% of hits, 5.6% overall), 102 crystals, 126.0 images/sec.
2125 images processed, 521 hits (24.5%), 129 indexable (24.8% of hits, 6.1% overall), 129 crystals, 58.2 images/sec.
2408 images processed, 606 hits (25.2%), 150 indexable (24.8% of hits, 6.2% overall), 150 crystals, 56.5 images/sec.
2781 images processed, 711 hits (25.6%), 178 indexable (25.0% of hits, 6.4% overall), 178 crystals, 74.4 images/sec.
3208 images processed, 795 hits (24.8%), 200 indexable (25.2% of hits, 6.2

Running for x=-511.0, y=-511.5


215 images processed, 73 hits (34.0%), 12 indexable (16.4% of hits, 5.6% overall), 12 crystals, 42.7 images/sec.
529 images processed, 152 hits (28.7%), 23 indexable (15.1% of hits, 4.3% overall), 23 crystals, 62.0 images/sec.
1005 images processed, 247 hits (24.6%), 43 indexable (17.4% of hits, 4.3% overall), 43 crystals, 94.9 images/sec.
1154 images processed, 330 hits (28.6%), 55 indexable (16.7% of hits, 4.8% overall), 55 crystals, 28.6 images/sec.
1691 images processed, 413 hits (24.4%), 77 indexable (18.6% of hits, 4.6% overall), 77 crystals, 107.4 images/sec.
2074 images processed, 498 hits (24.0%), 99 indexable (19.9% of hits, 4.8% overall), 99 crystals, 76.1 images/sec.
2335 images processed, 583 hits (25.0%), 129 indexable (22.1% of hits, 5.5% overall), 129 crystals, 51.5 images/sec.
2714 images processed, 677 hits (24.9%), 155 indexable (22.9% of hits, 5.7% overall), 155 crystals, 73.7 images/sec.
3051 images processed, 765 hits (25.1%), 180 indexable (23.5% of hits, 5.9% ov

Running for x=-511.0, y=-511.0


215 images processed, 73 hits (34.0%), 10 indexable (13.7% of hits, 4.7% overall), 10 crystals, 42.3 images/sec.
520 images processed, 148 hits (28.5%), 18 indexable (12.2% of hits, 3.5% overall), 18 crystals, 60.7 images/sec.
1004 images processed, 246 hits (24.5%), 39 indexable (15.9% of hits, 3.9% overall), 39 crystals, 95.3 images/sec.
1147 images processed, 329 hits (28.7%), 52 indexable (15.8% of hits, 4.5% overall), 52 crystals, 27.9 images/sec.
1711 images processed, 418 hits (24.4%), 65 indexable (15.6% of hits, 3.8% overall), 65 crystals, 111.9 images/sec.
2078 images processed, 501 hits (24.1%), 88 indexable (17.6% of hits, 4.2% overall), 88 crystals, 73.3 images/sec.
2353 images processed, 591 hits (25.1%), 104 indexable (17.6% of hits, 4.4% overall), 104 crystals, 54.6 images/sec.
2736 images processed, 691 hits (25.3%), 131 indexable (19.0% of hits, 4.8% overall), 131 crystals, 76.6 images/sec.
3065 images processed, 776 hits (25.3%), 152 indexable (19.6% of hits, 5.0% ov

Then based on RMSD between found peaks and indexed reflections combine the best indexings from xGandalf for each frame to one .stream file and then write a .sol from combinded .stream file.

In [ ]:
# Find Best Results Function Definitions

from best_results_def import find_best_results, read_stream_write_sol

In [ ]:
# Find Best Results
inputfolder_path = '/home/buster/hMTH1_TH287'
lattice = "oP" # Pearson Symbol read from .cell file first letter for lattice type and second for centering

##### Enter your parameters above #####

find_best_results(inputfolder_path)

import os

output_file_path = os.path.join(inputfolder_path, "best_results.stream")

read_stream_write_sol(output_file_path, lattice)

Integrate with best found ring for integration (From Fast Integration)

In [ ]:
# Full Integration Function Definitions

from full_int_def import run_indexamajig_with_frames

In [ ]:
# Full Integration
geomfile_path = ".geom"
cellfile_path = ".cell"
input_path = ""

x = -512
y = -512
num_threads = 23
indexing_method = "file"
resolution_push = 5
integration_method = "rings"
int_radius = "4,5,9"
min_peaks = 5
xgandalf_tolerance = 0.02
xgandalf_sampling_pitch = 5
xgandalf_min_vector_length = 40
xgandalf_max_vector_length = 110
xgandalf_iterations = 1
tolerance = "5,5,5,5"
overpredict = True

##### Enter your parameters above #####

run_indexamajig_with_frames(x, y, geomfile_path, cellfile_path, input_path, num_threads, indexing_method, resolution_push, integration_method, int_radius, min_peaks, xgandalf_tolerance, xgandalf_sampling_pitch, xgandalf_iterations, tolerance)

Merge integration results

In [ ]:
# Full Merge Function Definitions

from full_merge_def import merge_and_write_mtz

In [ ]:
# Merge
input_folder_path = ""
cellfile_path = ""
pointgroup = ""

##### Enter your parameters above #####

merge_and_write_mtz(input_folder_path, cellfile_path, pointgroup)

Refinement using REFMAC5

In [ ]:
# Full Refine Function Definitions

from full_ref_def import process_run_folders

In [ ]:
# Full Refine

run_number = 1
bins = 30

# Global variables for filenames
base_path = ""
pdb_filename = ".pdb"
mtz_orig_filename = "output.mtz"
output_filename = f"output_bins_{bins}.txt"

process_run_folders(base_path, run_number)